## Amazon API Gateway, AWS Lambda and Amazon DynamoDB

This example deploys an Amazon API Gateway, AWS Lambda functions and an Amazon DynamoDB table using the AWS CDK and TypeScript. The AWS Lambda functions are deployed behind an Amazon API Gateway and are used to execute CRUD operations against Amazon DynamoDB. The Amazon DynamoDB table is partitioned on an accountid attribute and also includes a sort key on the vendorid attribute, together they form the primary key. The example also demonstrates using Python to put, update, get and delete items in Amazon DynamoDB. A Lambda layer is deployed which includes a python module used to capture order details as metrics. The final part of the demonstration uses CloudWatch Log Insights to view the captured metrics.

![architecture](../images/architecture_1.png "Architecture")

**Jupyter Notebook Scripts**
1.	The *Post Order* script create orders and posts them to the Amazon API Gateway

2.	The *json formatter* script creates a json formatter object which is used to render the json results.

3.	The *Get Order* script passes query string parameters to a Lambda function using the API Gateway. The function returns items that match the parameters

4.	The *Update Order* script, updates multiple items in the table.

5.	The *Get Updated Order* script is used again to validate the update operation.

6.	The *Delete Order* script is used to delete items from the DynamoDB table.

7.	The *Get Order* script is used to validate the delete operation.

8.  The final part of the demonstration uses CloudWatch Log Insights to view the captured metrics.


**Note:** Make sure you set **gwid** to your gateway id using - *gwid = '...'*

**Post Order**

In [1]:
import boto3, requests, datetime
from random import randrange

#Set gateway id
gwid = '...'

city_list=["Adelaide","Brisbane","Canberra","Darwin","Geelong","Gold Coast","Hobart","Melbourne","Perth","Sydney","Wollongong"]
coffeetype_list=["Short Black","Flat White","Latte","Long Black"]
coffeesize_list=[{"size":"Small","price":"3.5"},{"size":"Medium","price":"4.0"},{"size":"Large","price":"4.5"},{"size":"x-Large","price":"5.0"}]

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/putorder')

for i in range(10): #By increasing this counter you will generate more insteresting statistics for log insights
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #Original value-----------
    coffeesize = randrange(4)
    #-------------------------

    quantity = randrange(10)+1
    city = city_list[randrange(11)]
    eventtype="new_order"
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype_list[randrange(4)],
                'coffeesize': coffeesize_list[coffeesize]["size"],
                'unitprice': coffeesize_list[coffeesize]["price"],
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    })

print(response)

<Response [200]>


**json formatter** - The following script create a class display nicely formatted json data

In [ ]:
import json, uuid
from IPython.display import display_javascript, display_html, display

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict) or isinstance(json_data, list):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;font: 12px/18px monospace !important;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
            renderjson.set_show_to_level(2);
            document.getElementById('%s').appendChild(renderjson(%s))
        });
      """ % (self.uuid, self.json_str), raw=True)

**Get Order** from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
get_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/getorder')

response_get = requests.get(get_url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

**Update Order**. Update the first 10 orders to Small Flat White.

In [ ]:
update_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/updateorder')


for i in range(10):
    accountid = 'a' + str(i)
    vendorid = 'v' + str(i)
    orderdate = str(datetime.datetime.now())
    
    #updated value------------
    coffeetype = 'Flat white'
    coffeesize = 'Small'
    #-------------------------
    
    unitprice = "4.50"
    quantity = 1
    city = city_list[i]
    eventtype="update_order"
    response = requests.post(url,json={'order':{
            'accountid': accountid,
            'orderdate':orderdate,
            'vendorid': vendorid,
            'city':city,
            'details':{
                'coffeetype': coffeetype,
                'coffeesize': coffeesize,
                'unitprice': unitprice,
                'quantity': quantity
            },
            'eventtype':[eventtype]
        }
    })

print(response)

**Get Updated Order** from Amazon DynamoDB using the primary key (accountid & vendorid)

In [ ]:
get_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/getorder')

response_get = requests.get(get_url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

**Delete Order** the order form Amazon DynamoDB

In [ ]:
delete_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/deleteorder')

response = requests.post(delete_url,json={'order':{'accountid':'a0','vendorid':'v0'}})

RenderJSON(response_get.json())

**Check Deleted** items in Amazon DynamoDB

In [ ]:
get_url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/getorder')

response_get = requests.get(get_url, params={'accountid':'a0','vendorid':'v0'})

RenderJSON(response_get.json())

**CloudWatch Log Insights**. Using CloudWatch Log Instight, filter the /aws/lambda/apigw_Lambda_dynamodb_put_item log with the following queries. Remember that the queries run against the log file not the table.

**Flat White count**
```
fields @timestamp, @message, coffeetype
| filter (coffeetype="Flat White")
| stats count(coffeetype)
```

**Flat White count in Hobart**
```
fields @timestamp, @message, coffeetype
| filter (coffeetype="Flat White") and (city="Hobart")
| stats count(coffeetype)
```

**Small coffee count and Samll avg**
```
fields @timestamp, @message, coffeetype, coffeesize
| filter (coffeesize="Small")
| stats count(coffeetype), avg(quantity)
```